In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
qubit = 3

## Qubit

In [3]:
from functools import reduce
from gen_qubits import PauliFrame

In [4]:
qubit = 3
pf = PauliFrame(3)
initial_hash = hash(pf)
cols = [ [f"Z{i}", f"X{i}", f"Y{i}"] for i in range(qubit)]
columns = reduce(lambda a, b: a+b, cols)
columns

['Z0', 'X0', 'Y0', 'Z1', 'X1', 'Y1', 'Z2', 'X2', 'Y2']

In [5]:
cols 

[['Z0', 'X0', 'Y0'], ['Z1', 'X1', 'Y1'], ['Z2', 'X2', 'Y2']]

In [6]:
df_frame_data = pd.DataFrame(columns = ["hash", *columns])
df_frame_connection_data = pd.DataFrame(columns = ["hash1", "hash2", "i", "j", "type"])

In [7]:
j=0


In [8]:
L = 0
LE = 0
for j in range(7):
    with open(f"qubit{qubit}_{j}_PauliFrame.data", "rb") as file:
        f = pickle.load(file)
    frames = list(f[1])
    
    print(j)
    
    for i, frame in enumerate(frames):
        arr = np.hstack([frame.matrix, np.bitwise_xor(frame.matrix[:, 0:2], frame.matrix[:, 2:])])
        frame_data = [(a,b) for a, b in zip(arr.flat[::2], arr.flat[1::2])]
        data = [hash(frame)] + frame_data 
        df_frame_data.loc[L] = data


        hash_step = [initial_hash] + [l[3] for l in frame.history]
        transofrmation = [l[:3] for l in frame.history]
        edge_map = [(hash_step[i], hash_step[i+1], *(transofrmation[i])) for i in range(len(hash_step)-1)]

        
        for e in edge_map:
            df_frame_connection_data.loc[LE] = e
            LE +=1
        L +=1
    
    print(j, "end")
    with open(f"qubit{qubit}_{j}_PauliFrame_ends.data", "rb") as file2:
        f2 = pickle.load(file2)
    frames2 = list(f2[1])
    for i, frame in enumerate(frames2):
        arr = np.hstack([frame.matrix, np.bitwise_xor(frame.matrix[:, 0:2], frame.matrix[:, 2:])])
        frame_data = [(a,b) for a, b in zip(arr.flat[::2], arr.flat[1::2])]
        data = [hash(frame)] + frame_data 
        df_frame_data.loc[L] = data
        
        hash_step = [initial_hash] + [l[3] for l in frame.history]
        transofrmation = [l[:3] for l in frame.history]
        edge_map = [(hash_step[i], hash_step[i+1], *(transofrmation[i])) for i in range(len(hash_step)-1)]

        for e in edge_map:
            df_frame_connection_data.loc[LE] = e
            LE +=1
        L +=1



0
0 end
1
1 end
2
2 end
3
3 end
4
4 end
5
5 end
6
6 end


In [9]:
df_frame_data.sort_values(by = columns, inplace=True)

In [18]:
df_frame_data.drop_duplicates(subset=["hash"], inplace=True)

In [19]:
df_frame_data

,hash,Z0,X0,Y0,Z1,X1,Y1,Z2,X2,Y2
3893,7934267287378752615,"(0, 1)","(1, 0)","(1, 1)","(0, 2)","(2, 2)","(2, 0)","(4, 4)","(4, 0)","(0, 4)"
9,-2543978279177338542,"(0, 1)","(1, 0)","(1, 1)","(0, 2)","(2, 4)","(2, 6)","(0, 4)","(4, 2)","(4, 6)"
7,-4465226254739119186,"(0, 1)","(1, 0)","(1, 1)","(0, 2)","(6, 0)","(6, 2)","(0, 6)","(4, 0)","(4, 6)"
5,727507648582393740,"(0, 1)","(1, 0)","(1, 1)","(0, 2)","(6, 4)","(6, 6)","(0, 6)","(4, 2)","(4, 4)"
3006,-8549410598421312604,"(0, 1)","(1, 0)","(1, 1)","(0, 4)","(4, 4)","(4, 0)","(0, 2)","(2, 0)","(2, 2)"
...,...,...,...,...,...,...,...,...,...,...
9160,5315319999447713569,"(7, 6)","(2, 0)","(5, 6)","(4, 1)","(4, 4)","(0, 5)","(7, 5)","(5, 4)","(2, 1)"
6274,5867513980184759253,"(7, 6)","(2, 5)","(5, 3)","(3, 7)","(0, 5)","(3, 2)","(7, 5)","(2, 1)","(5, 4)"
6854,3577838908738253112,"(7, 6)","(3, 6)","(4, 0)","(4, 1)","(7, 3)","(3, 2)","(0, 3)","(2, 1)","(2, 2)"
8884,-1143737878790462277,"(7, 7)","(0, 4)","(7, 3)","(1, 1)","(3, 6)","(2, 7)","(1, 7)","(2, 2)","(3, 5)"


In [26]:
df_frame_data.to_csv(r"df_frame_data_qubit3", header=True, index=None)
df_frame_connection_data.to_csv(r"df_frame_data_edge_qubit3", header=True, index=None)

In [21]:
df_frame_connection_data.sort_values(by=["hash1", "hash2"])

,hash1,hash2,i,j,type
23898,-9206615026885524136,487650781235202913,2,0,4
41988,-9206615026885524136,487650781235202913,0,2,4
63467,-9206615026885524136,487650781235202913,2,0,2
8668,-9206615026885524136,3151014126339180655,0,1,1
34778,-9206615026885524136,8016038942321655728,0,1,5
...,...,...,...,...,...
31247,9159861324757201004,7259727001935733252,2,0,2
9440,9159861324757201004,8549657490507590238,1,2,1
33173,9160794474742546931,-3086325893712175975,1,2,3
53118,9160794474742546931,-3086325893712175975,1,2,3


In [17]:
pf

1, IIZ, IIX
1, IZI, IXI
1, ZII, XII

In [25]:
(3384* (3384-1))

11448072